In [ ]:
import pandas as pd
import os
import torch
import numpy as np
from tqdm import trange
from deepscm.datasets.fundus import fundusDataset

ROOT_PATH = r'C:\Users\shiqi\PycharmProjects\DSCM_fundus\deepscm'
BASE_LOG_PATH = os.path.join(ROOT_PATH, r'assets\model\SVIExperiment')
version = 41
experiments = ['ConditionalVISEM']

In [ ]:
# load original fundus image
data_dir = '/mnt/beta/sliao/DSCM_fundus/deepscm/deepscm/datasets/split_dir3/test_features.xlsx'
base_path = '/mnt/alpha/diabetes/MS/data/images_512/'
downsample = 4
fundus_test = fundusDataset(data_dir,  base_path=base_path, crop_type='center', downsample=downsample)

In [ ]:
columns = [
    'exp', 'loss', 'log p(x)', 'log p(age)', 'log p(sex)', 'log p(T2D)',
     'log p(z) - log q(z)', 'p(z)', 'q(z)'
]
metrics_df = pd.DataFrame(columns=columns)

for exp in experiments:
    try:
        print(exp)
        metrics = torch.load(f'{BASE_LOG_PATH}/{exp}/version_{version}/metrics.pt')
        for k, v in metrics.items():
            print(f'{k}: {v:.03f}')
        row = {k.replace('test/', ''): np.array(v) for k, v in metrics.items()}
        metrics_df.loc[len(metrics_df)] = {'exp': exp, **row}
    except:
        pass

In [ ]:
recon_mae = {}
for exp in experiments:
    try:
        reconstructions = torch.load(f'{BASE_LOG_PATH}/{exp}/version_{version}/reconstruction.pt')
        diff = []
        for i in trange(reconstructions['x'].shape[0]):
            diff += [reconstructions['x'][i] - (fundus_test[i]['image'] * 255.)]
        recon_mae[exp] = np.mean(np.abs(torch.stack(diff).numpy()))
    except Exception as e:
        print(e)

metrics_df.loc[:, 'recon_mae'] = list(recon_mae.values())[0]

In [ ]:
metrics_df